<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_Single_Trial_Classification_NSL_2Class_Grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/EEG_Deep.git

Cloning into 'EEG_Deep'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 334 (delta 11), reused 0 (delta 0), pack-reused 313
Receiving objects: 100% (334/334), 1.69 GiB | 15.43 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Checking out files: 100% (94/94), done.


In [2]:
import scipy.io as spio
from google.colab import files
import numpy as np
import pandas as pd
import time
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet, EEGNet_Patient
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

!pip install neural_structured_learning
import tensorflow as tf
import neural_structured_learning as nsl

In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [11]:
from numpy import zeros
K.clear_session()
cols = 3
rows = 14
acc_all = zeros([rows, cols])
X_tr = np.empty([100, 15, 4096])
X_ts = np.empty([60, 15, 4096])
list_f_name=[]
h_cut = [24]
drop_out = [0.25]
k_len = [128] #[32, 64, 128]
n_epochs = 300

for h in h_cut:
  for d in drop_out:
    for kl in k_len:
      for x in range(1,15):
        fName = 'EEG_Deep/Data2A/Eopched_S0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['cleanRawEEGData']
        y_tr = mat['cleanClassLabels']
        y_tr = y_tr.flatten() 
  
        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
          tril = r_X_tr[t,:,:]
          # tril = tril.transpose()
          tril_filtered = butter_bandpass_filter(tril, lowcut=4, highcut=h, fs=250, order=4)
          # tril_filtered = tril_filtered.transpose()
          X_tr[t,:,:] = tril_filtered 
    
        # split data of each subject in training and validation
        X_train      = X_tr[0:70,:,1536:3072]
        Y_train      = y_tr[0:70]
        X_val       = X_tr[70:,:,1536:3072]
        Y_val       = y_tr[70:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))
  
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train-1)
        Y_val       = np_utils.to_categorical(Y_val-1)

        kernels, chans, samples = 1, 15, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val       = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        fName = 'EEG_Deep/Data2A/Eopched_S0' + str(x) + 'E.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_ts = mat['cleanRawEEGData']
        y_ts = mat['cleanClassLabels']
        y_ts = y_ts.flatten() 

        print(np.shape(r_X_ts))
        print(np.shape(y_ts))

        for t in range(r_X_ts.shape[0]):
          tril = r_X_ts[t,:,:]
          # tril = tril.transpose()
          tril_filtered = butter_bandpass_filter(tril, lowcut=4, highcut=h, fs=250, order=4)
          # tril_filtered = tril_filtered.transpose()
          X_ts[t,:,:] = tril_filtered 

        X_test      = X_ts[:,:,1536:3072]
        Y_test      = y_ts[:]
        print(np.shape(X_test))
        print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        Y_test      = np_utils.to_categorical(Y_test-1)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = EEGNet(nb_classes = 2, Chans = 15, Samples = 1536, 
                       dropoutRate = d, kernLength = kl, F1 = 8, 
                       D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')

        adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.5, adv_grad_norm='infinity')
        adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)
  
      # compile the model and set the optimizers
        adv_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
        batch_size = 32

        X_train = tf.cast(X_train, tf.float32)
        X_test = tf.cast(X_test, tf.float32)
        X_val = tf.cast(X_val, tf.float32)

        train_data = tf.data.Dataset.from_tensor_slices({'input': X_train, 'label': Y_train}).batch(batch_size)
        val_data = tf.data.Dataset.from_tensor_slices({'input': X_val, 'label': Y_val}).batch(batch_size)
        test_data = tf.data.Dataset.from_tensor_slices({'input': X_test, 'label': Y_test}).batch(batch_size)

        val_steps = X_val.shape[0] // batch_size
  
        history = adv_model.fit(train_data, validation_data=val_data, 
                                validation_steps=None, epochs=n_epochs, verbose=1)

        # # Plot training & validation accuracy values
        # plt.plot(history.history['categorical_accuracy'])
        # plt.plot(history.history['val_categorical_accuracy'])
        # plt.title('Model accuracy')
        # plt.ylabel('Accuracy')
        # plt.xlabel('Epoch')
        # plt.legend(['Train', 'Test'], loc='upper left')
        # plt.show()
        # figName = 'Accuracy_A0' + str(x) + '.png'  
        # plt.savefig(figName)

        print('\n# Evaluate on test data')
        results = adv_model.evaluate(test_data)
        print('test loss, test acc:', results)

        acc_all[x - 1, 0] = results[0]
        acc_all[x - 1, 1] = results[1]
        acc_all[x - 1, 2] = results[2]

        from keras import backend as K 
        # Do some code, e.g. train and save model
        K.clear_session()


      out_f_name = 'df_accl_h' + str(h) + '_d' + str(d) + '_k' + str(kl) +'_epochs_' + str(n_epochs) + '.csv'
      print(out_f_name)
      df_accl_all = pd.DataFrame({'loss': acc_all[:, 0], 'categorical_crossentropy': acc_all[:, 1],'categorical_accuracy': acc_all[:, 2]})
      print(df_accl_all)
      df_accl_all.to_csv (out_f_name, index = None, header=True) #Don't forget to add '.csv' at the end of the path
      # list_f_name.append(out_f_name)
      # time.sleep(10)
      # print('waiting for 10 sec')
      # files.download(out_f_name)


EEG_Deep/Data2A/Eopched_S01T.mat
(100, 15, 4096)
(100,)
(70, 15, 1536)
(70,)
(30, 15, 1536)
(30,)
X_train shape: (70, 1, 15, 1536)
70 train samples
30 val samples
EEG_Deep/Data2A/Eopched_S01E.mat
(60, 15, 4096)
(60,)
(60, 15, 1536)
(60,)
X_train shape: (60, 1, 15, 1536)
60 test samples
Train on 3 steps, validate on 1 steps
Epoch 1/300
3/3 [==============================] - 0s 119ms/step - loss: 0.7030 - categorical_crossentropy: 0.7030 - categorical_accuracy: 0.5429 - val_loss: 0.6914 - val_categorical_crossentropy: 0.6914 - val_categorical_accuracy: 0.6000
Epoch 2/300
3/3 [==============================] - 0s 20ms/step - loss: 0.6556 - categorical_crossentropy: 0.6556 - categorical_accuracy: 0.7714 - val_loss: 0.6901 - val_categorical_crossentropy: 0.6901 - val_categorical_accuracy: 0.5667
Epoch 3/300
3/3 [==============================] - 0s 20ms/step - loss: 0.6234 - categorical_crossentropy: 0.6234 - categorical_accuracy: 0.7286 - val_loss: 0.6889 - val_categorical_crossentropy: 0.